In [ ]:
# Imports and Setup
import os
import sys
import time
import json
from statistics import mean, stdev
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import numpy as np

# Load environment
load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI') or os.getenv('URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME') or os.getenv('USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD') or os.getenv('PASSWORD')

print(f"Connecting to: {NEO4J_URI}")

In [ ]:
# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver.verify_connectivity()
print("✅ Connected to Neo4j")

In [ ]:
# Load both embedding models
print("Loading embedding models...")

start = time.time()
model_minilm = SentenceTransformer('all-MiniLM-L6-v2')
load_time_minilm = time.time() - start
print(f"  ✅ MiniLM loaded in {load_time_minilm:.2f}s (384 dims)")

start = time.time()
model_mpnet = SentenceTransformer('all-mpnet-base-v2')
load_time_mpnet = time.time() - start
print(f"  ✅ MPNet loaded in {load_time_mpnet:.2f}s (768 dims)")

## 1. Embedding Speed Comparison

Compare how fast each model generates embeddings for text of varying lengths.

In [ ]:
# Test queries of varying complexity
test_queries = [
    "delays at LAX",
    "What are the top airports with delays?",
    "How do Millennials travel compared to Baby Boomers in terms of destinations and aircraft preferences?",
    "Which aircraft type has the best on-time performance and lowest average arrival delays for business class passengers?",
    "Compare the food satisfaction scores for premier gold loyalty members flying on Boeing 737-800 versus Airbus A320 on routes from Los Angeles to New York with more than 2000 miles traveled."
]

print("Embedding Speed Comparison (10 runs each)\n" + "="*60)

embedding_results = []

for query in test_queries:
    # MiniLM
    minilm_times = []
    for _ in range(10):
        start = time.time()
        _ = model_minilm.encode(query)
        minilm_times.append(time.time() - start)
    
    # MPNet
    mpnet_times = []
    for _ in range(10):
        start = time.time()
        _ = model_mpnet.encode(query)
        mpnet_times.append(time.time() - start)
    
    result = {
        "query": query[:50] + "..." if len(query) > 50 else query,
        "query_length": len(query),
        "minilm_avg_ms": mean(minilm_times) * 1000,
        "mpnet_avg_ms": mean(mpnet_times) * 1000,
        "speedup": mean(mpnet_times) / mean(minilm_times)
    }
    embedding_results.append(result)
    
    print(f"\nQuery ({len(query)} chars): {query[:50]}...")
    print(f"  MiniLM: {result['minilm_avg_ms']:.2f}ms")
    print(f"  MPNet:  {result['mpnet_avg_ms']:.2f}ms")
    print(f"  → MiniLM is {result['speedup']:.1f}x faster")

## 2. Retrieval Quality Comparison

Compare the top-K results from each model for the same queries.

In [ ]:
def semantic_search_journeys(driver, query_embedding, index_name, top_k=10):
    """Run vector search on Journey nodes."""
    search_query = f"""
    CALL db.index.vector.queryNodes('{index_name}', $top_k, $query_embedding)
    YIELD node, score
    MATCH (p:Passenger)-[:TOOK]->(node)-[:ON]->(f:Flight)
    OPTIONAL MATCH (f)-[:DEPARTS_FROM]->(o:Airport)
    OPTIONAL MATCH (f)-[:ARRIVES_AT]->(d:Airport)
    RETURN node.passenger_class AS passenger_class,
           node.arrival_delay_minutes AS delay,
           node.actual_flown_miles AS miles,
           p.generation AS generation,
           p.loyalty_program_level AS loyalty,
           f.fleet_type_description AS fleet,
           o.station_code AS origin,
           d.station_code AS destination,
           score
    ORDER BY score DESC
    LIMIT $top_k
    """
    with driver.session() as session:
        result = session.run(search_query, top_k=top_k, query_embedding=query_embedding)
        return [dict(r) for r in result]

In [ ]:
# Test retrieval with diverse queries
retrieval_queries = [
    {
        "query": "flights with long delays from Los Angeles",
        "expected_keywords": ["LAX", "delay"]
    },
    {
        "query": "business class passengers with excellent food service",
        "expected_keywords": ["Business", "food"]
    },
    {
        "query": "millennial travelers on long distance routes",
        "expected_keywords": ["Millennial", "miles"]
    },
    {
        "query": "premier gold loyalty members with on-time arrivals",
        "expected_keywords": ["premier gold", "delay"]
    },
    {
        "query": "Boeing 737 aircraft performance",
        "expected_keywords": ["B737", "737"]
    }
]

print("Retrieval Quality Comparison (Top-10 results)\n" + "="*60)

retrieval_results = []

In [ ]:
for test in retrieval_queries:
    query = test["query"]
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print("="*60)
    
    # Generate embeddings
    emb_minilm = model_minilm.encode(query).tolist()
    emb_mpnet = model_mpnet.encode(query).tolist()
    
    # Search with MiniLM
    start = time.time()
    try:
        results_minilm = semantic_search_journeys(driver, emb_minilm, "journey_embedding_minilm", top_k=10)
        search_time_minilm = (time.time() - start) * 1000
    except Exception as e:
        print(f"  ⚠️ MiniLM search failed: {e}")
        results_minilm = []
        search_time_minilm = 0
    
    # Search with MPNet
    start = time.time()
    try:
        results_mpnet = semantic_search_journeys(driver, emb_mpnet, "journey_embedding_mpnet", top_k=10)
        search_time_mpnet = (time.time() - start) * 1000
    except Exception as e:
        print(f"  ⚠️ MPNet search failed: {e}")
        results_mpnet = []
        search_time_mpnet = 0
    
    # Compare results
    print(f"\n📊 MiniLM Results ({len(results_minilm)} found, {search_time_minilm:.1f}ms):")
    for i, r in enumerate(results_minilm[:5], 1):
        print(f"  {i}. score={r['score']:.3f} | {r.get('origin','?')}->{r.get('destination','?')} | {r.get('fleet','?')} | {r.get('generation','?')} | delay={r.get('delay',0)}min")
    
    print(f"\n📊 MPNet Results ({len(results_mpnet)} found, {search_time_mpnet:.1f}ms):")
    for i, r in enumerate(results_mpnet[:5], 1):
        print(f"  {i}. score={r['score']:.3f} | {r.get('origin','?')}->{r.get('destination','?')} | {r.get('fleet','?')} | {r.get('generation','?')} | delay={r.get('delay',0)}min")
    
    # Calculate similarity score statistics
    if results_minilm:
        minilm_scores = [r['score'] for r in results_minilm]
        minilm_avg = mean(minilm_scores)
        minilm_max = max(minilm_scores)
    else:
        minilm_avg = minilm_max = 0
        
    if results_mpnet:
        mpnet_scores = [r['score'] for r in results_mpnet]
        mpnet_avg = mean(mpnet_scores)
        mpnet_max = max(mpnet_scores)
    else:
        mpnet_avg = mpnet_max = 0
    
    retrieval_results.append({
        "query": query,
        "minilm_search_time_ms": search_time_minilm,
        "mpnet_search_time_ms": search_time_mpnet,
        "minilm_avg_score": minilm_avg,
        "mpnet_avg_score": mpnet_avg,
        "minilm_max_score": minilm_max,
        "mpnet_max_score": mpnet_max,
        "minilm_results_count": len(results_minilm),
        "mpnet_results_count": len(results_mpnet)
    })
    
    print(f"\n📈 Score Statistics:")
    print(f"  MiniLM: avg={minilm_avg:.3f}, max={minilm_max:.3f}")
    print(f"  MPNet:  avg={mpnet_avg:.3f}, max={mpnet_max:.3f}")

## 3. Summary & Recommendations

In [ ]:
print("\n" + "="*70)
print("EMBEDDING MODEL COMPARISON SUMMARY")
print("="*70)

# Model loading times
print(f"\n📦 Model Loading Time:")
print(f"  MiniLM (384 dims): {load_time_minilm:.2f}s")
print(f"  MPNet  (768 dims): {load_time_mpnet:.2f}s")

# Embedding speed
if embedding_results:
    avg_minilm = mean([r['minilm_avg_ms'] for r in embedding_results])
    avg_mpnet = mean([r['mpnet_avg_ms'] for r in embedding_results])
    avg_speedup = mean([r['speedup'] for r in embedding_results])
    
    print(f"\n⚡ Embedding Speed (avg across queries):")
    print(f"  MiniLM: {avg_minilm:.2f}ms")
    print(f"  MPNet:  {avg_mpnet:.2f}ms")
    print(f"  → MiniLM is {avg_speedup:.1f}x faster")

# Search latency
if retrieval_results:
    valid_minilm = [r['minilm_search_time_ms'] for r in retrieval_results if r['minilm_search_time_ms'] > 0]
    valid_mpnet = [r['mpnet_search_time_ms'] for r in retrieval_results if r['mpnet_search_time_ms'] > 0]
    
    if valid_minilm and valid_mpnet:
        avg_search_minilm = mean(valid_minilm)
        avg_search_mpnet = mean(valid_mpnet)
        
        print(f"\n🔍 Vector Search Latency (Neo4j):")
        print(f"  MiniLM: {avg_search_minilm:.1f}ms")
        print(f"  MPNet:  {avg_search_mpnet:.1f}ms")

# Similarity scores
if retrieval_results:
    valid_minilm_scores = [r['minilm_avg_score'] for r in retrieval_results if r['minilm_avg_score'] > 0]
    valid_mpnet_scores = [r['mpnet_avg_score'] for r in retrieval_results if r['mpnet_avg_score'] > 0]
    
    if valid_minilm_scores and valid_mpnet_scores:
        avg_score_minilm = mean(valid_minilm_scores)
        avg_score_mpnet = mean(valid_mpnet_scores)
        
        print(f"\n🎯 Average Similarity Scores:")
        print(f"  MiniLM: {avg_score_minilm:.3f}")
        print(f"  MPNet:  {avg_score_mpnet:.3f}")

print(f"\n" + "="*70)
print("RECOMMENDATIONS")
print("="*70)
print("""
✅ Use MiniLM (all-MiniLM-L6-v2) when:
   - Speed is critical (real-time applications)
   - Memory/storage is constrained
   - Good-enough quality is acceptable

✅ Use MPNet (all-mpnet-base-v2) when:
   - Quality is the top priority
   - Latency tolerance is higher
   - Complex semantic matching is needed

💡 For this Graph-RAG application:
   - MiniLM is recommended as default (good balance)
   - MPNet available as option for quality-focused queries
   - User can switch in UI sidebar based on needs
""")

In [ ]:
# Save results to JSON
comparison_data = {
    "model_info": {
        "minilm": {
            "name": "all-MiniLM-L6-v2",
            "dimensions": 384,
            "load_time_s": load_time_minilm
        },
        "mpnet": {
            "name": "all-mpnet-base-v2",
            "dimensions": 768,
            "load_time_s": load_time_mpnet
        }
    },
    "embedding_speed_results": embedding_results,
    "retrieval_results": retrieval_results
}

with open('embedding_comparison_results.json', 'w') as f:
    json.dump(comparison_data, f, indent=2)

print("✅ Results saved to embedding_comparison_results.json")

In [ ]:
# Create comparison table
print("\n" + "="*80)
print("COMPARISON TABLE")
print("="*80)
print(f"{'Metric':<35} {'MiniLM (384d)':<20} {'MPNet (768d)':<20}")
print("-"*80)
print(f"{'Model Load Time':<35} {f'{load_time_minilm:.2f}s':<20} {f'{load_time_mpnet:.2f}s':<20}")

if embedding_results:
    print(f"{'Avg Embedding Time':<35} {f'{avg_minilm:.2f}ms':<20} {f'{avg_mpnet:.2f}ms':<20}")

if retrieval_results and valid_minilm and valid_mpnet:
    print(f"{'Avg Search Latency':<35} {f'{avg_search_minilm:.1f}ms':<20} {f'{avg_search_mpnet:.1f}ms':<20}")

if retrieval_results and valid_minilm_scores and valid_mpnet_scores:
    print(f"{'Avg Similarity Score':<35} {f'{avg_score_minilm:.3f}':<20} {f'{avg_score_mpnet:.3f}':<20}")

print(f"{'Vector Dimensions':<35} {'384':<20} {'768':<20}")
print(f"{'Index Size (relative)':<35} {'1x (smaller)':<20} {'2x (larger)':<20}")
print("="*80)

In [ ]:
# Close connection
driver.close()
print("\n✅ Neo4j connection closed")